In [1]:
company_summary = '''
이 회사는 기업을 위한 SaaS(Software as a Service) 솔루션을 제공하는 IT 기업으로, 주로 경영 지원 시스템의 비효율성을 해결하는 데 중점을 두고 있습니다. 회사의 주요 제품과 서비스는 다음과 같습니다:

### 문제점
1. **비효율적인 경영 지원 시스템**: 많은 기업들이 여전히 수작업으로 반복적인 업무를 처리하고 있으며, 이는 전체 업무 시간의 40% 이상을 차지합니다.
2. **커뮤니케이션 오버헤드**: 팀 간의 비효율적인 커뮤니케이션으로 인해 추가적인 시간과 비용이 발생합니다.
3. **자산 관리의 어려움**: 유형 자산 및 무형 자산의 관리가 비효율적으로 이루어지고 있습니다.

### 솔루션
1. **SaaS 기반 경영 지원 시스템**: 
   - **자동화**: 반복적인 업무를 자동화하여 효율성을 높입니다.
   - **AI 에이전트**: AI 기반의 에이전트를 통해 실시간으로 데이터를 분석하고 인사이트를 제공합니다.
   - **자산 관리**: QR 코드 기반의 자산 관리 시스템을 통해 자산의 효율적인 관리가 가능합니다.

2. **커뮤니케이션 효율화**:
   - **통합 커뮤니케이션 플랫폼**: 다양한 커뮤니케이션 도구를 통합하여 오버헤드를 줄입니다.
   - **데이터 기반 인사이트**: 커뮤니케이션 데이터를 분석하여 효율성을 높이는 전략을 제안합니다.

3. **데이터 인사이트 제공**:
   - **실시간 데이터 분석**: 기업 데이터를 실시간으로 분석하여 경영 인사이트를 제공합니다.
   - **리스크 및 유효 기간 관리**: 자산의 유효 기간을 관리하고 만료 알림을 제공합니다.

### 비즈니스 모델
- **서비스 제공**: 국내 기업에 SaaS 및 솔루션을 제공하며, 임직원 서비스도 함께 제공합니다.
- **수익 모델**: 월 사용료 및 라이선스 비용을 통해 수익을 창출합니다. 다양한 플랜(베이직, 비즈니스, 엔터프라이즈)을 통해 고객의 필요에 맞춘 서비스를 제공합니다.

### 시장 및 목표
- **시장 확장**: 글로벌 빅데이터 및 분석 솔루션 시장으로의 확장을 목표로 하고 있습니다.
- **매출 목표**: 2030년까지 매출 11000억 원 달성을 목표로 하고 있습니다.

### 팀 구성
- **CEO**: 유민재
- **CTO**: 김종민
- **개발자**: 이승현

이 회사는 IT 자산 관리 및 데이터 분석 솔루션을 통해 기업의 경영 효율성을 높이고, 글로벌 시장으로의 확장을 목표로 하고 있습니다.
'''

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from pprint import pprint

# ───────────────────────────────────────
# 환경 설정
# ───────────────────────────────────────
load_dotenv()
OPENAI_MODEL    = os.getenv("OPENAI_MODEL", "gpt-4.1-mini")
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY  = os.getenv("TAVILY_API_KEY")   # Tavily 웹 검색 API

# ───────────────────────────────────────
# 상태 정의
# ───────────────────────────────────────
class HybridState(TypedDict):
    company_info: Dict[str, Any]                         # 입력된 회사 요약
    planner_response:   Annotated[List, add_messages]    # 키워드 추출 결과
    llm_response:       Annotated[List, add_messages]    # LLM이 생성한 리드
    tavily_response:    Annotated[List, add_messages]    # Tavily 검색으로 가져온 리드
    merge_response:     Annotated[List, add_messages]    # 병합된 리드 리스트
    final_report:       Annotated[List, add_messages]    # 최종 리포트(JSON)

# 헬퍼: 마지막 메시지 추출
latest = lambda st, k: st[k][-1] if st[k] else None

# ───────────────────────────────────────
# 1. Planner: 회사 요약에서 키워드 추출
# ───────────────────────────────────────
planner_prompt = (
    "<역할> B2B 세일즈 타게팅 플래너\n"
    "<목표> 회사 요약에서 keywords, exclusion, location을 JSON으로 추출\n"
    "반드시 JSON 형식만 출력"
)

def planner_node(state: HybridState) -> HybridState:
    summary = state["company_info"]["summary"]
    llm     = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    resp    = llm.invoke([{"role":"system","content": planner_prompt + "\n" + summary}])
    try:
        plan = json.loads(resp.content)
    except:
        plan = {"keywords": [], "exclusion": [], "location": "대한민국"}
    state["planner_response"].append(
       
       HumanMessage(content=json.dumps(plan, ensure_ascii=False))
    )
    return state

# ───────────────────────────────────────
# 2. LLM Generator: 키워드로 리드 생성
# ───────────────────────────────────────
def llm_generator_node(state: HybridState) -> HybridState:
    plan = json.loads(latest(state, "planner_response").content)
    kws  = ", ".join(plan.get("keywords", []))
    prompt = f"""
<역할> 세일즈 전문가
<키워드> {kws}
대한민국 소재, 경쟁사 제외. 아래 스키마로 리드 10개를 JSON 배열로 작성:
[
  {{"companyName":"","industryKeywords":[],"homepage":"","location":"",
    "founderOrCEO":"","founded":"","summary":"","targetCustomer":""}}
]
"""
    llm  = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    resp = llm.invoke([{"role":"system","content": prompt}])
    try:
        leads = json.loads(resp.content)
    except:
        leads = []
    state["llm_response"].append(
        HumanMessage(content=json.dumps(leads, ensure_ascii=False))
    )
    return state

# ───────────────────────────────────────
# 3. Tavily Search: Tavily API로 추가 리드 탐색 (디버깅 포함)
# ───────────────────────────────────────
def tavily_search_node(state: HybridState) -> HybridState:
    plan = json.loads(latest(state, "planner_response").content)
    headers = {"Authorization": f"Bearer {TAVILY_API_KEY}", "Content-Type": "application/json"}
    tavily_leads = []

    # LLM이 생성한 리드 목록을 활용하여 회사명별로 디버그 쿼리 수행
    llm_leads = json.loads(latest(state, "llm_response").content)
    for lead in llm_leads:
        company_name = lead.get("companyName", "").strip()
        for kw in plan.get("keywords", []):
            query = f'"{company_name}" "{kw}" 협력 사례 2024'
            print(f"[DEBUG][tavily_search] QUERY: {query}")
            try:
                r = requests.post(
                    "https://api.tavily.com/search",
                    headers=headers,
                    json={"query": query, "max_results": 5},
                    timeout=10
                )
                print(f"[DEBUG][tavily_search] HTTP 상태 코드: {r.status_code}")
                data = r.json()
                results = data.get("results", [])
                print(f"[DEBUG][tavily_search] 결과 개수: {len(results)}")
                for item in results:
                    title = item.get("title", "").split("|")[0].strip()
                    url   = item.get("url", "")
                    snippet = item.get("content", "")[:120]
                    tavily_leads.append({
                        "companyName": company_name,
                        "title":       title,
                        "snippet":     snippet,
                        "url":         url
                    })
            except Exception as e:
                print(f"[ERROR][tavily_search] API 호출 실패: {e}")

    # 중복 제거: (회사명, URL) 기준
    unique = { (l['companyName'], l['url']): l for l in tavily_leads }.values()
    unique_list = list(unique)
    print(f"[DEBUG][tavily_search] 고유 리드 개수: {len(unique_list)}")

    state["tavily_response"].append(
        HumanMessage(content=json.dumps(unique_list, ensure_ascii=False))
    )
    return state

# ───────────────────────────────────────
# 4. Merge: LLM 및 Tavily 결과 병합 (중복 제거)
# ───────────────────────────────────────
def merge_node(state: HybridState) -> HybridState:
    llm_leads   = json.loads(latest(state, "llm_response").content)
    tavily_leads= json.loads(latest(state, "tavily_response").content)
    seen = {lead["companyName"] for lead in llm_leads}
    new_leads = [l for l in tavily_leads if l["companyName"] not in seen]
    merged = llm_leads + new_leads
    state["merge_response"].append(
        HumanMessage(content=json.dumps(merged, ensure_ascii=False))
    )
    return state

# ───────────────────────────────────────
# 5. Reporter: 최종 리포트 출력
# ───────────────────────────────────────
def reporter_node(state: HybridState) -> dict:
    final = json.loads(latest(state, "merge_response").content)
    state["final_report"].append(
        HumanMessage(content=json.dumps(final, ensure_ascii=False))
    )
    return {"final_report": state["final_report"]}

# ───────────────────────────────────────
# 그래프 정의 및 실행
# ───────────────────────────────────────
def run():
    g = StateGraph(HybridState)
    g.add_node("planner",         planner_node)
    g.add_node("llm_generator",  llm_generator_node)
    g.add_node("tavily_search",   tavily_search_node)
    g.add_node("merge",           merge_node)
    g.add_node("reporter",        reporter_node)

    g.set_entry_point("planner")
    g.add_edge("planner",        "llm_generator")
    g.add_edge("llm_generator",  "tavily_search")
    g.add_edge("tavily_search",   "merge")
    g.add_edge("merge",          "reporter")
    g.set_finish_point("reporter")

    app = g.compile()

    init_state: HybridState = {
        "company_info":      {"summary": company_summary},
        "planner_response":  [],
        "llm_response":      [],
        "tavily_response":   [],
        "merge_response":    [],
        "final_report":      []
    }

    app = g.compile()

    init_state: HybridState = {
        "company_info":      {"summary": "<회사 요약 텍스트>"},
        "planner_response":  [],
        "llm_response":      [],
        "tavily_response":   [],
        "merge_response":    [],
        "final_report":      []
    }

    for delta in app.stream(init_state):
        for node_name, output in delta.items():
            pprint(f"노드 '{node_name}' 실행됨")
            if node_name == "reporter":
                data = output.get("final_report", [])
                if data:
                    print("\n--- 최종 리드 리스트 ---\n")
                    print(data[-1].content)
        pprint("\n---\n")

if __name__ == "__main__":
    run()



"노드 'planner' 실행됨"
'\n---\n'
"노드 'llm_generator' 실행됨"
'\n---\n'
[DEBUG][tavily_search] 고유 리드 개수: 0
"노드 'tavily_search' 실행됨"
'\n---\n'
"노드 'merge' 실행됨"
'\n---\n'
"노드 'reporter' 실행됨"

--- 최종 리드 리스트 ---

[{"companyName": "에코그린솔루션", "industryKeywords": ["친환경", "재생에너지", "태양광"], "homepage": "http://www.ecogreensolution.co.kr", "location": "서울특별시 강남구", "founderOrCEO": "김민수", "founded": "2015-03-12", "summary": "태양광 패널 및 재생에너지 솔루션을 제공하는 친환경 에너지 기업입니다.", "targetCustomer": "중소기업 및 공공기관"}, {"companyName": "스마트헬스케어코리아", "industryKeywords": ["헬스케어", "웨어러블", "의료기기"], "homepage": "http://www.smarthealthcare.kr", "location": "서울특별시 마포구", "founderOrCEO": "이수진", "founded": "2018-07-01", "summary": "웨어러블 의료기기 개발 및 헬스케어 데이터 분석 서비스를 제공합니다.", "targetCustomer": "병원 및 개인 건강관리 사용자"}, {"companyName": "모바일핀테크", "industryKeywords": ["핀테크", "모바일결제", "블록체인"], "homepage": "http://www.mobilefintech.co.kr", "location": "경기도 성남시 분당구", "founderOrCEO": "박준형", "founded": "2017-11-15", "summary": "모바일 결제 및 블록체인 기반 금융 서비스 플